In [13]:
import cv2
import numpy as np
import os
import glob
from tqdm import tqdm

def extract_protein_and_concentration(dir_path):
    """
    Extracts the protein and DNA concentration from the directory path.
    Assumes that the folder immediately preceding 'Rep1' is of the form 'Protein_Concentration'
    (e.g. 'AdPa_160nM').
    """
    norm_path = os.path.normpath(dir_path)
    parts = norm_path.split(os.sep)
    try:
        rep_index = parts.index("Rep1")
    except ValueError:
        raise ValueError(f"Directory {dir_path} does not contain 'Rep1'.")
    if rep_index == 0:
        raise ValueError(f"Directory {dir_path} is not structured properly to extract protein info.")
    protein_conc = parts[rep_index - 1]
    tokens = protein_conc.split('_')
    if len(tokens) != 2:
        raise ValueError(f"Expected folder name in the form 'Protein_Concentration', got {protein_conc}.")
    return protein_conc

def synchronize_image_sequences(video_data, time_multiplier, grid_shape, channel,
                                fps_output=30.0, output_file_path="combined_video.avi"):
    """
    Synchronizes multiple image sequences (directories containing .jpg images) along a common global time axis,
    annotates each cell with:
      - A timer (top left) showing the global synchronized time (in minutes and hours),
      - The protein/DNA info (top right),
      - The file name (bottom left),
      - And a scale bar (bottom right) where 730 pixels correspond to 1 mm.
    The annotated cells are arranged into a grid in a single output video.
    
    Parameters:
      video_data (list of tuples): Each tuple is (dir_path, effective_frame_duration in seconds).
      time_multiplier (float): Multiplier applied to the minimum effective frame duration to determine the global time step.
      grid_shape (tuple): (rows, columns) for the output grid.
      channel (str): Channel name appended to the output filename.
      fps_output (float): Playback frames per second for the output video.
      output_file_path (str): A file path whose directory is used for saving the output video.
      
    Returns:
      None. Saves the output video and prints its playback duration.
    """
    # Unzip the list of tuples.
    dir_paths, frame_durations = zip(*video_data)
    n_sequences = len(dir_paths)
    grid_cells = grid_shape[0] * grid_shape[1]
    if grid_cells < n_sequences:
        raise ValueError("Grid shape is too small for the number of image sequences provided.")
    
    # Extract protein/DNA info for naming and annotation.
    protein_infos = [extract_protein_and_concentration(dp) for dp in dir_paths]
    proteins_combined = "-".join(protein_infos)
    
    # Build final output file name.
    out_dir = os.path.dirname(output_file_path)
    final_output_file = os.path.join(out_dir, f"{proteins_combined}_{channel}_synced.avi")
    
    # Gather image files, dimensions, and total durations.
    image_files_list = []
    num_frames_list = []
    dimensions = []
    total_times = []
    
    for dp, fd in zip(dir_paths, frame_durations):
        files = sorted(glob.glob(os.path.join(dp, "*.jpg")))
        if not files:
            raise ValueError(f"No .jpg images found in directory: {dp}")
        image_files_list.append(files)
        num_frames = len(files)
        num_frames_list.append(num_frames)
        total_times.append(num_frames * fd)
        # Read first image to determine dimensions.
        first_img = cv2.imread(files[0])
        if first_img is None:
            raise ValueError(f"Failed to read the first image in directory: {dp}")
        h, w = first_img.shape[:2]
        dimensions.append((h, w))
    
    # Use the longest sequence duration to cover all sequences.
    total_time_sync = max(total_times)
    base_time_step = min(frame_durations)
    time_step = base_time_step * time_multiplier
    num_output_frames = int(total_time_sync / time_step)
    
    playback_duration_sec = num_output_frames / fps_output
    print(f"Output video will be {playback_duration_sec:.2f} seconds long.")
    
    # Define a common cell size.
    cell_height = max(h for h, w in dimensions)
    cell_width  = max(w for h, w in dimensions)
    output_frame_height = grid_shape[0] * cell_height
    output_frame_width  = grid_shape[1] * cell_width
    
    # Create the video writer.
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(final_output_file, fourcc, fps_output, (output_frame_height, output_frame_width))
    
    # Process each global time step.
    for i in tqdm(range(num_output_frames), desc="Processing frames", unit="frame"):
        current_time = i * time_step
        cell_frames = []  # To store annotated cell frames
        
        for idx, (files, fd, n_frames) in enumerate(zip(image_files_list, frame_durations, num_frames_list)):
            frame_idx = int(current_time / fd)
            if frame_idx >= n_frames:
                frame_idx = n_frames - 1
            frame = cv2.imread(files[frame_idx])
            if frame is None:
                frame = np.zeros((dimensions[idx][0], dimensions[idx][1], 3), dtype=np.uint8)
            frame = cv2.resize(frame, (cell_width, cell_height))
            
            # --- Annotations using FONT_HERSHEY_SIMPLEX (stand-in for Arial) ---
            # 1. Timer (top left), moved further down with larger text.
            time_min = current_time / 60.0
            time_hr  = current_time / 3600.0
            timer_text_min = f"{time_min:.2f} min"
            timer_text_hr = f"{time_hr:.2f} hr"
            cv2.putText(frame, timer_text_min, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (255,255,255), 3)
            cv2.putText(frame, timer_text_hr, (10, 130), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (255,255,255), 3)
            
            # 2. Protein & DNA info (top right), moved down with larger text.
            info_text = protein_infos[idx].replace("_", " ")
            text_size, _ = cv2.getTextSize(info_text, cv2.FONT_HERSHEY_SIMPLEX, 2.0, 3)
            cv2.putText(frame, info_text, (cell_width - text_size[0] - 10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 2.0, (255,255,255), 3)
            
            # 3. File name (bottom left)
            file_name = os.path.basename(files[frame_idx])
            cv2.putText(frame, file_name, (10, cell_height - 30), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 3)
            
            # 4. Scale bar (bottom right); 730 pixels = 1 mm.
            bar_length = 730
            bar_thickness = 4
            bar_end = (cell_width - 10, cell_height - 30)
            bar_start = (cell_width - 10 - bar_length, cell_height - 30)
            cv2.line(frame, bar_start, bar_end, (255,255,255), bar_thickness)
            scale_text = "1 mm"
            text_size, _ = cv2.getTextSize(scale_text, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 3)  # Increased text size
            text_x = bar_start[0] + (bar_length - text_size[0]) // 2
            text_y = bar_start[1] - 15  # Adjusted position for larger text
            cv2.putText(frame, scale_text, (text_x, text_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 3)  # Increased text size
            
            cell_frames.append(frame)
        
        while len(cell_frames) < grid_cells:
            black = np.zeros((cell_height, cell_width, 3), dtype=np.uint8)
            cell_frames.append(black)
        
        grid_rows = []
        for r in range(grid_shape[0]):
            row_frames = cell_frames[r * grid_shape[1] : (r + 1) * grid_shape[1]]
            row_combined = np.hstack(row_frames)
            grid_rows.append(row_combined)
        combined_frame = np.vstack(grid_rows)
        out.write(combined_frame)
    
    out.release()
    print(f"Output saved as: {final_output_file}")

# --- Example usage ---

video_data = [
    # (directory_path, effective_frame_duration in seconds)
    (f"../../../../Thomson Lab Dropbox/David Larios/activedrops/main/030225-AdPa-titrations/2p5ulTMB_1ulDNA_/AdPa_160nM/Rep1/piv_movie/", 30*18),
    (f"../../../../Thomson Lab Dropbox/David Larios/activedrops/main/030225-AdPa-titrations/2p5ulTMB_1ulDNA_/AdPa_80nM/Rep1/piv_movie/", 30*18),
    (f"../../../../Thomson Lab Dropbox/David Larios/activedrops/main/030225-AdPa-titrations/2p5ulTMB_1ulDNA_/AdPa_40nM/Rep1/piv_movie/", 30*18),
    (f"../../../../Thomson Lab Dropbox/David Larios/activedrops/main/030225-AdPa-titrations/2p5ulTMB_1ulDNA_/AdPa_20nM/Rep1/piv_movie/", 30*18),
    (f"../../../../Thomson Lab Dropbox/David Larios/activedrops/main/030225-AdPa-titrations/2p5ulTMB_1ulDNA_/AdPa_10nM/Rep1/piv_movie/", 30*18),
    (f"../../../../Thomson Lab Dropbox/David Larios/activedrops/main/030225-AdPa-titrations/2p5ulTMB_1ulDNA_/AdPa_5nM/Rep1/piv_movie/", 30*18),
    (f"../../../../Thomson Lab Dropbox/David Larios/activedrops/main/030225-AdPa-titrations/2p5ulTMB_1ulDNA_/AdPa_2p5nM/Rep1/piv_movie/", 30*18),
    (f"../../../../Thomson Lab Dropbox/David Larios/activedrops/main/030225-AdPa-titrations/2p5ulTMB_1ulDNA_/AdPa_1p25nM/Rep1/piv_movie/", 30*12),
]

grid_shape = (2, 4)
time_multiplier = 4
output_file_path = "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/all/movies/combined_video_placeholder.avi"

synchronize_image_sequences(video_data, time_multiplier, grid_shape, channel="piv",
                            fps_output=60, output_file_path=output_file_path)

Output video will be 1.50 seconds long.


Processing frames: 100%|██████████| 90/90 [00:08<00:00, 10.09frame/s]

Output saved as: ../../../../Thomson Lab Dropbox/David Larios/activedrops/main/all/movies/AdPa_160nM-AdPa_80nM-AdPa_40nM-AdPa_20nM-AdPa_10nM-AdPa_5nM-AdPa_2p5nM-AdPa_1p25nM_piv_synced.avi


In [16]:

video_data = [
    # (directory_path, effective_frame_duration in seconds)
    (f"../../../../Thomson Lab Dropbox/David Larios/activedrops/main/101324-k401-titration-rt/2p5TMB-1ulDNA_/K401_160nM-RT/Rep1/piv_movie/", 60),
    (f"../../../../Thomson Lab Dropbox/David Larios/activedrops/main/101324-k401-titration-rt/2p5TMB-1ulDNA_/K401_80nM-RT/Rep1/piv_movie/", 60*2),
    (f"../../../../Thomson Lab Dropbox/David Larios/activedrops/main/101324-k401-titration-rt/2p5TMB-1ulDNA_/K401_40nM-RT/Rep1/piv_movie/", 60*5),
    (f"../../../../Thomson Lab Dropbox/David Larios/activedrops/main/101324-k401-titration-rt/2p5TMB-1ulDNA_/K401_20nM-RT/Rep1/piv_movie/", 60*8),
]

grid_shape = (1, 4)
time_multiplier = 8
output_file_path = "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/all/movies/combined_video_placeholder.avi"

synchronize_image_sequences(video_data, time_multiplier, grid_shape, channel="piv",
                            fps_output=1, output_file_path=output_file_path)

Output video will be 326.00 seconds long.


Processing frames: 100%|██████████| 326/326 [00:21<00:00, 15.18frame/s]

Output saved as: ../../../../Thomson Lab Dropbox/David Larios/activedrops/main/all/movies/K401_160nM-RT-K401_80nM-RT-K401_40nM-RT-K401_20nM-RT_piv_synced.avi
